In [1]:
# 连接kafka 需要安装
# pip install kafka-python
# pip install lz4
import pymysql
from sqlalchemy import create_engine
import pandas as pd 


/opt/anaconda3/lib/python3.9/site-packages/pandas/core/computation/expressions.py:21: UserWarning: Pandas requires version '2.8.4' or newer of 'numexpr' (version '2.7.3' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED
/opt/anaconda3/lib/python3.9/site-packages/pandas/core/arrays/masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.2' currently installed).
  from pandas.core import (


In [2]:
# 链接测试数据库，读，目前数据库数据全空
# DB配置 kafka配置
host = 
user = 
passwd = 
db = 
port = 
config = {
            "host": host,
            "user": user,
            "passwd": passwd,
            "db": db,
            "port": port        
        }


In [7]:
# DB读写
link = f"mysql+pymysql://{config['user']}:{config['passwd']}@{config['host']}:{config['port']}/{config['db']}"
cnn = create_engine(link)
connect = pymysql.connect(
    host=config['host'],
    user=config['user'],
    passwd=config['passwd'],
    db=config['db'],
    port=config['port'],
    charset="utf8",
    autocommit=True)
cur = connect.cursor()



# 读
selectsql = f"select  *  FROM anomaly_1_result  "
cur.execute(selectsql)
results = cur.fetchall()
column_names = [description[0] for description in cur.description]
data_df = pd.DataFrame(results, columns=column_names)
print(data_df.to_markdown())

# 写 从csv读数据到pandas df然后给到db
#df_minute_api = pd.read_csv(r'test.csv')
# try:
#     df_minute_api.to_sql('anomaly_task1_result', cnn, if_exists='append',index=False)
#     print("插入数据库成功！！！")
# except:
#     print("插入数据库失败！！！")



| id   | anomaly_code   | sub_code   | sub_exchange   | sub_name   | anomaly_type   | anomaly_ind_code   | anomaly_date   | anomaly_time   | url   | title   | content   | company   | perspective   | attribution_dimension   | sentiment   | create_time   | trending   | industry   | stock   |
|------|----------------|------------|----------------|------------|----------------|--------------------|----------------|----------------|-------|---------|-----------|-----------|---------------|-------------------------|-------------|---------------|------------|------------|---------|


In [8]:
# 将data转成json ,为了给kafka发送数据
json_str = data_df.to_json(orient='records')
print(json_str)

[]


kafka的topic  
1.任务1  sjs_task_one  
2.任务2  sjs_task_two  
3.任务3  sjs_task_three  
4.任务4  sjs_task_four  

In [13]:
# Kafka 生产者
from kafka import KafkaProducer
from kafka.errors import KafkaConfigurationError
 
# Kafka 配置信息
kafka_hosts = [''] 
sasl_username = ''      
sasl_password = ''       

# SASL_PLAINTEXT 认证配置
security_protocol = 'SASL_PLAINTEXT'
sasl_mechanism = 'SCRAM-SHA-256'

# 创建 Kafka 生产者
def create_kafka_producer(hosts, security_protocol, sasl_mechanism, sasl_username, sasl_password):
    try:
        producer = KafkaProducer(
            bootstrap_servers=hosts,
            security_protocol=security_protocol,
            sasl_mechanism=sasl_mechanism,
            sasl_plain_username=sasl_username,
            sasl_plain_password=sasl_password
        )
        return producer
    except KafkaConfigurationError as e:
        print(f"Error creating Kafka producer: {e}")

# 使用配置创建 Kafka 生产者实例
producer = create_kafka_producer(
    hosts=kafka_hosts,
    security_protocol=security_protocol,
    sasl_mechanism=sasl_mechanism,
    sasl_username=sasl_username,
    sasl_password=sasl_password
)
 
# 检查生产者是否创建成功
if producer:
    print("Kafka producer created successfully.")
else:
    print("Failed to create Kafka producer.")


Kafka producer created successfully.


In [14]:
# 发送数据到Kafka topic
# 将json_str编码为字节串
value_bytes = json_str.encode('utf-8')
future = producer.send('sjs_task_one',  value=value_bytes)

# 等待发送完成（可选，但通常在生产环境中，你会想要异步发送）
result = future.get(timeout=60)

print(f'Message sent to sjs_task_one partition {result.partition} offset {result.offset}')

# 关闭生产者连接 
producer.close()

Message sent to sjs_task_one partition 0 offset 2


In [18]:
# kafka 消费者
from kafka import KafkaConsumer
from kafka.errors import KafkaError
 
# Kafka 配置信息
kafka_hosts = ['']  
sasl_username = ''       
sasl_password = ''       
 
# SASL_PLAINTEXT 认证配置
security_protocol = 'SASL_PLAINTEXT'
sasl_mechanism = 'SCRAM-SHA-256'
 
# 需要消费的 topic
topic = 'sjs_task_one'
 
# 创建消费者
consumer = KafkaConsumer(
    topic,
    bootstrap_servers=kafka_hosts,
    security_protocol=security_protocol,
    sasl_mechanism=sasl_mechanism,
    sasl_plain_username=sasl_username,
    sasl_plain_password=sasl_password,
    auto_offset_reset='earliest'   # 配置消费模式 从最早的消息开始消费
)

# 检查消费者是否创建成功
if consumer:
    print("Kafka consumer created successfully.")
else:
    print("Failed to create Kafka consumer.")

# 循环读取消息
for message in consumer:
    try:
        # 打印接收到的消息
        print(message.value.decode('utf-8'))
        #break #如果不想循环的话就break

    except KafkaError as e:
        # 处理错误
        print(f"Kafka error: {e}")



Kafka consumer created successfully.
[{"id":1,"anomaly_code":"1","sub_code":"1","sub_exchange":"1","sub_name":"1","anomaly_type":"1","anomaly_ind_code":"1","anomaly_date":"1","anomaly_time":1722211200000,"url":"1","title":"1","content":"1","company":"1","perspective":"1","attribution_dimension":"1","sentiment":"1","create_time":1722211200000}]
[]
[]


KeyboardInterrupt: 